## Векторизация состава почвы на ходу по проектной территории из tiff

In [ ]:
gdf_tiles.to_file('test.gpkg')

In [ ]:
gdf_tiles

In [ ]:
import tifffile as tiff

with tiff.TiffFile('example_data/russia_tiles/tile_0_0_2456126_4564077/clay/clay_0-5cm_mean.tif') as tif:
        page = tif.pages[0]
        tags = page.tags
        
tags.values()

## Векторизация ландшафта на ходу по проектной территории из tiff

In [ ]:
from ecodonut.landscape_vectorizer import vectorize_height_isolines_for_zone, vectorize_height_polygons_for_zone, \
    vectorize_aspect_polygons_for_zone, vectorize_slope_polygons_for_zone
import geopandas as gpd
from pathlib import Path

zone = gpd.read_file("example_data/project Шлиссельбург.geojson").to_crs(32636)
tiles = gpd.read_parquet("example_data/fabdem_tiles.parquet")  # файл с тайлами
tiff_dir = Path("example_data")  # папка с FABDEM.tiff 

# iso_lines = vectorize_height_isolines_for_zone(zone_gdf=zone,tiles_gdf=tiles,tiffs_dir=tiff_dir,height_step=1,smooth_sigma=0.1)
heigh_map = vectorize_height_polygons_for_zone(zone_gdf=zone,tiles_gdf=tiles,tiffs_dir=tiff_dir,height_step=1,smooth_sigma=0.1)
slope_map = vectorize_slope_polygons_for_zone(zone_gdf=zone,tiles_gdf=tiles,tiffs_dir=tiff_dir,slope_step_deg=1,smooth_sigma=0.1)
aspect_map = vectorize_aspect_polygons_for_zone(zone_gdf=zone,tiles_gdf=tiles,tiffs_dir=tiff_dir,aspect_step_deg=45,smooth_sigma=0.1)


In [ ]:
from matplotlib.colors import LinearSegmentedColormap


def make_aspect_cmap(name: str = "aspect360") -> LinearSegmentedColormap:
    stops = [
        (0.00, "#2b83ba"),  # N   (0°)    blue
        (0.25, "#1ec66d"),  # E   (90°)   green
        (0.50, "#d7191c"),  # S   (180°)  red
        (0.75, "#9c89b8"),  # W   (270°)  purple
        (1.00, "#2b83ba"),  # N   (360°)  blue (wrap)
    ]
    return LinearSegmentedColormap.from_list(name, stops)


cmap = make_aspect_cmap('aspect_deg')
aspect_map.to_crs(3857).plot(column='aspect_deg', cmap=cmap)

In [ ]:
m1 = heigh_map.explore(column='height', tiles='cartodb positron')
slope_map.explore(m=m1, column='slope_deg', cmap='Reds')

## Работа с векторизованными слоями по тайлам 

In [ ]:
from pathlib import Path
import geopandas as gpd
from ecodonut.landscape_vectorizer import (
    stitch_height_isolines, stitch_height_polygons,
    stitch_slope_polygons, stitch_aspect_polygons,
)

zone = gpd.read_file("example_data/project_zone.geojson")
zone.geometry = zone.to_crs(zone.estimate_utm_crs()).geometry.buffer(5000).to_crs(zone.crs)  # Буффер, если надо
tiles = gpd.read_parquet("example_data/fabdem_tiles.parquet")  # файл с тайлами
vectors_dir = Path("D://fabdem_tiles_russia")  # папка с векторизованными слоями

# 1) Изолинии
iso = stitch_height_isolines(zone, tiles, vectors_dir, height_step=5)

# 2) Полигоны высоты
highpoly = stitch_height_polygons(zone, tiles, vectors_dir, height_step=5)

# 3) Уклон
slope = stitch_slope_polygons(zone, tiles, vectors_dir, slope_step_deg=2)

# 4) Экспозиция
aspect = stitch_aspect_polygons(zone, tiles, vectors_dir, aspect_step_deg=90)